In [1]:
import os
from dotenv import load_dotenv
_ = load_dotenv()

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import AzureChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun

In [2]:
# Azure AI Foundry configuration
model = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-12-01-preview",
    azure_deployment="gpt-4.1",  # Your deployment name
    model="gpt-4.1",
    temperature=0
)

In [3]:
model.invoke([SystemMessage(content="Ready for working?")]).content

"Yes, I'm ready to help! What do you need assistance with today?"

## Import Prompts

In [4]:
import sys
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Load the intent classifier prompt from the text file
with open(os.path.join(parent_dir, "prompts", "intent_classifier.txt"), "r") as f:
    intent_classifier_prompt = f.read()

intent_classifier_prompt

'You are an intent classifier for the ChatSupervisor agent, the central brain of a worker assistant. Your primary responsibility is to analyze the user\'s latest message in the context of the chat history and determine the user\'s primary intent.\n\nYour response must be only the name of the single, most appropriate agent or tool to handle this intent. Do not provide any other text, explanation, or preamble. Respond with the agent name on a single line, with no extra whitespace, punctuation, or formatting.\n\nThese are your only valid routing options:\n\n1.  InformationRetrievalAgent\n    - Purpose: To answer generic questions about work policies, procedures, or "how-to" guides using the internal knowledge base.\n    - Examples:\n        - "How do I call in sick?"\n        - "What are the company rules for overtime?"\n        - "How are reservations calculated?"\n\n2.  ActionExecutionAgent\n    - Purpose: To handle personalized requests that require fetching the user\'s own specific da

## Agent Set-up

In [38]:
# Agent State
class AgentState(TypedDict):
   messages: Annotated[list[AnyMessage], "Conversation messages"]
   next_action: Annotated[str, "Next action to take"]
   retrieved_data: Annotated[str, "Data retrieved from tools"]
   error_message: Annotated[str, "Error message if any"]
   attempts: Annotated[int, "Number of attempts made"]
   max_retries: Annotated[int, "Maximum number of retries allowed"]

In [ ]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END

# Ensure AgentState is defined (as discussed previously)
# class AgentState(TypedDict):
#     messages: Annotated[List[BaseMessage], operator.add]
#     next_action: str
#     retrieved_data: str
#     error: str

class ChatSupervisorAgent:
    def __init__(self, model, checkpointer=None):
        self.model = model
        self.system = intent_classifier_prompt 

        graph = StateGraph(AgentState)

        # --- 1. Define Nodes ---
        graph.add_node("classify_intent", self.classify_intent_node)
        graph.add_node("action_execution_agent", self.action_execution_agent)
        graph.add_node("information_retrieval_agent", self.information_retrieval_agent)
        graph.add_node("fallback_tool", self.fallback_tool)
        graph.add_node("answer_agent", self.answer_agent) # REQUIRED by architecture

        # --- 2. Define The Entry Point ---
        graph.set_entry_point("classify_intent")

        # --- 3. Define The Conditional Edge (The Router) ---
        graph.add_conditional_edges(
            "classify_intent", 
            self.route_intent, 
            {
                "ActionExecutionAgent": "action_execution_agent",
                "InformationRetrievalAgent": "information_retrieval_agent",
                "FallbackTool": "fallback_tool",
            }
        )

        # --- 4. Define Normal Edges (Enforcing ToV) ---
        graph.add_edge("action_execution_agent", "answer_agent")
        graph.add_edge("information_retrieval_agent", "answer_agent")
        graph.add_edge("fallback_tool", "answer_agent")
        
        # AnswerAgent finishes the run
        graph.add_edge("answer_agent", END)

        # 5. Compile
        self.graph = graph.compile(checkpointer=checkpointer)

    def classify_intent_node(self, state: AgentState):
        '''Reads the user query and classifies the primary action.'''
        messages = [SystemMessage(content=self.system)] + state['messages']
        response = self.model.invoke(messages)
        intent = response.content.strip()

        # Failsafe for valid routing
        valid_routes = ["ActionExecutionAgent", "InformationRetrievalAgent", "FallbackTool"]
        if intent not in valid_routes:
            print(f"Warning: Invalid intent '{intent}'. Defaulting to Fallback.")
            intent = "FallbackTool"

        print(f"Intent classified as: {intent}")
        
        # Return the update to the state
        return {"next_action": intent}

    def route_intent(self, state: AgentState):
        '''Reads the decision made by the intent classifier node'''
        return state["next_action"]

    # --- SPECIALISTS ---
    def action_execution_agent(self, state: AgentState):
        '''...'''
        print("---Start Action Execution Agent---")
        # Logic to call your tools...
        # return {"retrieved_data": "Schedule for next week..."}
        return {"retrieved_data": "Success placeholder"}

    def information_retrieval_agent(self, state: AgentState):
        '''Retrieves information from the Sharepoint KnowledgeBase'''
        print("---Start Information Retrieval Agent---")
        # Logic to call RAG...
        return {"retrieved_data": "Sick leave policy details..."}

    def fallback_tool(self, state: AgentState):
        print("---Start Fallback Tool---")
        return {"error": "Could not resolve query."}

    # --- ANSWER AGENT (Final Response) ---
    def answer_agent(self, state: AgentState):
        print("---Start Answer Agent---")
        # return {"messages": [AIMessage(content="Final response to user")]}
        return {}

In [89]:
agent = ChatSupervisorAgent(model=model)

In [90]:
# message = "what's my schedule for next week?"
message = "How can I call in sick?"
# message = "What's my availability next week?"
# message = "What's Trump's first name?"


In [91]:
test_state = {
    "messages": [HumanMessage(content=message)],
    "next_action": "",
    "retrieved_data": "",
    "error_message": "",
    "attempts": 0,
    "max_retries": 3
}

result = agent.graph.invoke(test_state)

Intent classified as: InformationRetrievalAgent
---Start Information Retrieval Agent---
---Start Answer Agent---


In [92]:
result

{'messages': [HumanMessage(content='How can I call in sick?', additional_kwargs={}, response_metadata={})],
 'next_action': 'InformationRetrievalAgent',
 'retrieved_data': 'Sick leave policy details...',
 'error_message': '',
 'attempts': 0,
 'max_retries': 3}